In [ ]:
import pandas as pd
import numpy as npc
import yfinance as yf
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8") 
%config InlineBackend.figure_format = "svg" # Better picture quality for plots
np.set_printoptions(suppress=True) # Removes -e numbers

In [ ]:
# Importing tickers, last 7 years for example
btc = yf.Ticker("BTC-USD").history(period = "7y")
eth = yf.Ticker("ETH-USD").history(period = "7y")

In [ ]:
# plt.plot(btc.index, btc["Close"], label = "BTC-USD");

In [ ]:
# plt.plot(eth.index, eth["Close"], label = "ETH-USD");

In [ ]:
# Comparing both assets, normalized - each close price divided by first one
plt.plot(btc.index, btc["Close"] / btc["Close"].iloc[0], label = "BTC Normalized"); 
plt.plot(eth.index, eth["Close"] / eth["Close"].iloc[0], label = "ETH Normalized");
plt.title("BTC vs ETH Growth past 7 years in a Normalized format")
plt.xlabel("Date")
plt.legend();

In [ ]:
# %(Simple) Returns
btc["Pct Return"] = btc["Close"].pct_change() 
eth["Pct Return"] = eth["Close"].pct_change() 

In [ ]:
# log returns
btc["Log Return"] = np.log(btc["Close"] / btc["Close"].shift(1))
eth["Log Return"] = np.log(eth["Close"] / eth["Close"].shift(1))

In [ ]:
# Cumulative Returns from log
btc["Cum Log Returns"] = np.exp(btc["Log Return"].cumsum()) - 1
eth["Cum Log Returns"] = np.exp(eth["Log Return"].cumsum()) - 1

In [ ]:
# Cumulative Returns from % Returns
btc["Cum Pct Return"] = (1 + btc["Pct Return"]).cumprod() - 1
eth["Cum Pct Return"] = (1 + eth["Pct Return"]).cumprod() - 1

In [ ]:
plt.plot(btc["Cum Log Returns"], label = "BTC Cumulative Return");
plt.plot(eth["Cum Log Returns"], label = "ETH Cumulative Return");
plt.legend();

In [ ]:
# Rolling and Annualized Volatility 
btc["Rolling_vol"] = btc["Log Return"].rolling(window=30).std()  # 30 trading days ~ 1 month
btc["Annualized_vol"] = btc["Rolling_vol"] * np.sqrt(365)  # 365 trading days in a year


eth["Rolling_vol"] = btc["Log Return"].rolling(window=30).std()  # 30 trading days ~ 1 month
eth["Annualized_vol"] = btc["Rolling_vol"] * np.sqrt(365)  # 365 trading days in a year

In [ ]:
# Sharpe Ratio - Measures risk-adjusted return
sharpe_btc = btc["Log Return"].mean() / btc["Log Return"].std() * np.sqrt(365)
sharpe_eth = eth["Log Return"].mean() / eth["Log Return"].std() * np.sqrt(365)

In [ ]:
# print(sharpe_btc) Sharpe ratio for the last 7 years of BTC

In [ ]:
# print(sharpe_eth)Sharpe ratio for the last 7 years of ETH

In [ ]:
# Simple Backtest Strategy of both SMA's crossing
# Fast(50) SMA crosses above Slow(200) SMA → Buy, indicating uptrend.
# Fast(50) SMA crosses below Slow(200) SMA → Sell, indicating downtrend

In [ ]:
# Simple Moving Averages
btc["SMA_50"] = btc["Close"].rolling(window = 50).mean()
btc["SMA_200"] = btc["Close"].rolling(window = 200).mean()

btc = btc.dropna(subset=["SMA_50", "SMA_200"]).copy() # Removing the first 200 for the backtest, since no 200 SMA is formed yet.


eth["SMA_50"] = eth["Close"].rolling(window = 50).mean()
eth["SMA_200"] = eth["Close"].rolling(window = 200).mean()

eth = eth.dropna(subset=["SMA_50", "SMA_200"]).copy()

In [ ]:
btc["Signal"] = 0 # Neutral, no position
btc.loc[btc['SMA_50'] > btc['SMA_200'], 'Signal'] = 1 # Long / Buy (entry / exit)
btc.loc[btc['SMA_50'] < btc['SMA_200'], 'Signal'] = -1 # Short / Sell (entry / exit)

In [ ]:
# Strategy return = previous day's signal * today's log return
btc["Strategy Return"] = btc["Signal"].shift(1) * btc["Log Return"]

In [ ]:
# Calculate the cumulative returns
btc["Cumulative Strategy"] = np.exp(btc["Strategy Return"].cumsum())  # start from 1
btc["Cumulative BuyHold"] = np.exp(btc["Log Return"].cumsum())  # for comparison 

In [ ]:
# Comparing the strategy to a simple Buy & Hold BTC one
plt.plot(btc["Cumulative BuyHold"], label = "Buy & Hold BTC", color = "blue")
plt.plot(btc["Cumulative Strategy"], label = "SMA 50/200 Strategy", color = "green")
plt.title("SMA Crossover Backtest")
plt.xlabel("Date")
plt.ylabel("Growth of $1")
plt.legend()

In [ ]:
# Calculate the sharpe ratio of the strategy
strategy_sharpe = btc["Strategy Return"].dropna().mean() / btc["Strategy Return"].dropna().std() * np.sqrt(365)

In [ ]:
# Print both the SMA Strategy Sharpe and BTC Buy % Hold strategy Sharpe
print("SMA Strategy Sharpe:", strategy_sharpe, "\nBTC Buy / Hold Sharpe:", sharpe_btc)

In [ ]:
# Drawdown Calculation - % decline from the previous peak of the equity curve.
# MDD (Max Drawdown) - Worst or deepest drawdown in the equity curve. 

In [ ]:
# SMA Strategy  Drawdown
btc["SMA Strategy Peak"] = btc["Cumulative Strategy"].cummax()
btc["SMA Strategy Drawdown"] = (btc["Cumulative Strategy"] - btc["SMA Strategy Peak"]) / btc["SMA Strategy Peak"]

In [ ]:
# Buy & Hold Strategy Drawdown
btc["BuyHold Strategy Peak"] = btc["Cumulative BuyHold"].cummax()
btc["BuyHold Strategy Drawdown"] = (btc["Cumulative BuyHold"] - btc["BuyHold Strategy Peak"]) / btc["BuyHold Strategy Peak"]

In [ ]:
# MDD
max_dd_sma = btc["SMA Strategy Drawdown"].min()
max_dd_buyhold = btc["BuyHold Strategy Drawdown"].min()

print("Max Drawdown (SMA Strategy):", max_dd_sma, "\nMax Drawdown (Buy & Hold):", max_dd_buyhold)

In [ ]:
# Plotting the drawdowns
plt.plot(btc["SMA Strategy Drawdown"], label="SMA Strategy Drawdown", color="black", linestyle = "-");
plt.plot(btc["BuyHold Strategy Drawdown"], label="Buy & Hold Drawdown", color="red", linewidth = 0.7, linestyle = "--");
plt.title("Drawdowns Over Time")
plt.xlabel("Date")
plt.ylabel("Drawdown (%)")
plt.legend();